In [ ]:
import json
import pandas as pd
from datasets import load_dataset

In [8]:
## DATASET PREPARATION

In [4]:
annotations = json.load(open("/Utilisateurs/umushtaq/emorec_work/mdlt_er/datasets/emoart5k/annotation.json", "r"))

In [9]:
annotations[0].keys()

dict_keys(['request_id', 'description', 'image_path'])

In [10]:
annotations[0]

{'request_id': 'Abstract Art_request-1',
 'description': {'first_section': {'description': 'The image features a playful arrangement of colored dots and abstract shapes scattered across a neutral background. The use of vibrant oranges, blues, and blacks offers a lively, rhythmic pattern that catches the eye.'},
  'second_section': {'visual_attributes': {'brushstroke': 'Smooth, blending seamlessly into the background.',
    'color': 'Warm and cool tones are used with moderate saturation, creating a harmonious contrast.',
    'composition': 'Dynamic and scattered, with no central focus, giving a sense of movement.',
    'light_and_shadow': 'Soft, even lighting with minimal contrast between light and dark areas.',
    'line_quality': 'The lines are smooth and defined, enhancing the playful feel.'},
   'emotional_impact': 'The combination of smooth brushstrokes, balanced colors, dynamic composition, and soft lighting evokes a cheerful, uplifting mood, promoting feelings of contentment and 

In [11]:
def flatten_entry(entry):
    desc = entry.get("description", {})

    first_desc = (
        desc.get("first_section", {})
            .get("description", None)
    )

    visual = (
        desc.get("second_section", {})
            .get("visual_attributes", {})
    )

    emotional = desc.get("third_section", {})

    return {
        "request_id": entry.get("request_id"),

        # main description
        "description": first_desc,

        # visual attributes
        "brushstroke": visual.get("brushstroke"),
        "color": visual.get("color"),
        "composition": visual.get("composition"),
        "light_and_shadow": visual.get("light_and_shadow"),
        "line_quality": visual.get("line_quality"),

        # emotional info
        "emotional_impact": desc.get("second_section", {}).get("emotional_impact"),
        "emotional_arousal_level": emotional.get("emotional_arousal_level"),
        "emotional_valence": emotional.get("emotional_valence"),
        "dominant_emotion": emotional.get("dominant_emotion"),
        "healing_effects": ", ".join(emotional.get("healing_effects", [])),

        # path
        "image_path": entry.get("image_path"),
    }


In [13]:
flat_data = [flatten_entry(d) for d in annotations]
df = pd.DataFrame(flat_data)
df.head()

,request_id,description,brushstroke,color,composition,light_and_shadow,line_quality,emotional_impact,emotional_arousal_level,emotional_valence,dominant_emotion,healing_effects,image_path
0,Abstract Art_request-1,The image features a playful arrangement of co...,"Smooth, blending seamlessly into the background.",Warm and cool tones are used with moderate sat...,"Dynamic and scattered, with no central focus, ...","Soft, even lighting with minimal contrast betw...","The lines are smooth and defined, enhancing th...","The combination of smooth brushstrokes, balanc...",Low,Positive,Calm,"Relieve Stress, Embrace Joy",Images\Abstract Art\0006080_YuriZlotnikov-Sign...
1,Abstract Art_request-2,"The artwork features a grid of vibrant, geomet...","Fine and precise, creating defined edges betwe...",Warm and cool tones intermingle; moderately sa...,"Dynamic with an irregular grid, providing bala...","Minimal shadowing, focused on flat color appli...","Thick and bold, creating a stark separation be...",The bold colors and structured composition evo...,High,Positive,Excited,"Gain Inspiration, Embrace Joy",Images\Abstract Art\0006185_PaulKlee-GlassFaca...
2,Abstract Art_request-3,The image features an abstract geometric compo...,"The brushwork is smooth and even, emphasizing ...",The palette includes warm and cool tones with ...,"The arrangement is dynamic, with a central foc...","There is minimal light and shadow, with flat c...","Lines are predominantly hard and smooth, contr...",The structured composition and harmonious colo...,Low,Positive,Calm,"Relieve Stress, Gain Inspiration",Images\Abstract Art\0006313_WilliBaumeister-Pl...
3,Abstract Art_request-4,The image features dynamic abstract forms in b...,"Fine and precise, emphasizing sharp edges and ...",Dominantly cool tones with stark black and mut...,Asymmetrical with a dynamic flow; elements gui...,"High contrast of black against light, enhancin...","Both thick and fine lines, with smooth, delibe...",The interplay of shapes and lines generates a ...,High,Positive,Excited,"Gain Inspiration, Embrace Joy",Images\Abstract Art\0006400_OttoFreundlich-Kom...
4,Abstract Art_request-5,The painting features an array of abstract sha...,"Smooth and subtle, blending seamlessly to form...","A mix of cool and warm tones, with moderate sa...","Dynamic, with a scattering of shapes that guid...",Subtle light sources diffuse across the canvas...,"Soft and flowing, yet defined, giving each sha...","The smooth transitions, harmonious color palet...",Low,Positive,Calm,"Relieve Stress, Find Hope",Images\Abstract Art\0006526_AlexejvonJawlensky...


In [14]:
df.shape

(5532, 13)

In [25]:
def process_paths(row):
    
    path_full = "/Utilisateurs/umushtaq/emorec_work/mdlt_er/datasets/emoart5k/" + row.image_path
    path = path_full.replace("\\", "/")
    return path

In [26]:
process_paths(df.iloc[0])

'/Utilisateurs/umushtaq/emorec_work/mdlt_er/datasets/emoart5k/Images/Abstract Art/0006080_YuriZlotnikov-Signalseries.jpg'

In [29]:
df["image_path_full"] = df.apply(lambda x: process_paths(x), axis=1)

In [31]:
df.iloc[0]["image_path_full"]

'/Utilisateurs/umushtaq/emorec_work/mdlt_er/datasets/emoart5k/Images/Abstract Art/0006080_YuriZlotnikov-Signalseries.jpg'

In [34]:
df['dominant_emotion'].unique()

array(['Calm', 'Excited', 'Contentment', 'Frustrated', 'Sad', 'Aroused',
       'Alarmed', 'Bored', 'Happy', 'Annoyed', 'Tired', 'Glad'],
      dtype=object)

In [32]:
### BUILD PROMPTS

In [58]:
def generation_instruction(row):
    
    
   #  emotion_classes = ['Calm', 'Excited', 'Contentment', 'Frustrated', 'Sad', 'Aroused',
   #     'Alarmed', 'Bored', 'Happy', 'Annoyed', 'Tired', 'Glad']
   #  formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])

   instruction = f"""You are an expert art analyst specializing in emotional interpretation of visual artwork. Your task is to analyze paintings and drawings and identify the single dominant emotion they evoke in a viewer.

Analyze the emotional content of this artwork and identify the single most dominant emotion it conveys or evokes.

Choose exactly one emotion from the following list:
['Calm', 'Excited', 'Contentment', 'Frustrated', 'Sad', 'Aroused', 'Alarmed', 'Bored', 'Happy', 'Annoyed', 'Tired', 'Glad']

Consider the following when making your assessment:
- Color palette and tone (warm/cool, saturated/muted)
- Composition and visual tension
- Subject matter and imagery
- Brushwork or line quality (chaotic, gentle, rigid, etc.)
- Overall mood the artwork projects

Respond only with a JSON object in this exact format:
{{"dominant_emotion": "<emotion>"}}

Do not include any explanation, commentary, or additional fields.

   """

   return instruction


In [59]:
def build_output(row):
        
    return {"dominant_emotion": row.dominant_emotion}

In [60]:
df["instruction"] = df.apply(lambda x: generation_instruction(x), axis=1)

In [61]:
df["answer"] = df.apply(lambda x: build_output(x), axis=1) # type: ignore

In [62]:
print(df.iloc[0]['instruction'])

You are an expert art analyst specializing in emotional interpretation of visual artwork. Your task is to analyze paintings and drawings and identify the single dominant emotion they evoke in a viewer.

Analyze the emotional content of this artwork and identify the single most dominant emotion it conveys or evokes.

Choose exactly one emotion from the following list:
['Calm', 'Excited', 'Contentment', 'Frustrated', 'Sad', 'Aroused', 'Alarmed', 'Bored', 'Happy', 'Annoyed', 'Tired', 'Glad']

Consider the following when making your assessment:
- Color palette and tone (warm/cool, saturated/muted)
- Composition and visual tension
- Subject matter and imagery
- Brushwork or line quality (chaotic, gentle, rigid, etc.)
- Overall mood the artwork projects

Respond only with a JSON object in this exact format:
{"dominant_emotion": "<emotion>"}

Do not include any explanation, commentary, or additional fields.

   


In [63]:
print(df.iloc[0]['answer'])

{'dominant_emotion': 'Calm'}


In [64]:
df.columns

Index(['request_id', 'description', 'brushstroke', 'color', 'composition',
       'light_and_shadow', 'line_quality', 'emotional_impact',
       'emotional_arousal_level', 'emotional_valence', 'dominant_emotion',
       'healing_effects', 'image_path', 'image_path_full', 'instruction',
       'answer'],
      dtype='object')